In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
pd.set_option('display.float_format','{:.2f}'.format)

In [ ]:
all_data = pd.read_parquet('Data/data_gl_id_mapped_2025_cleaned_v2.parquet')

In [ ]:
all_data.columns

In [ ]:
all_data.rename(columns={
    'total_sale_amount' : 'SALE_AMT',
    'e_sale_amount' : 'E_SALE_AMT'
},inplace=True)

In [ ]:
cond = (all_data['global_id'] == all_data['STO_FIN_HRCHY__DCSTAR'])
all_data['flag_gl_id'] = np.select([cond],[1],default=0)

In [ ]:
all_data['global_id'].isna().sum()

In [ ]:
all_data.columns

In [ ]:
all_data['Non-E_SALE_AMT'] = all_data['SALE_AMT'] - all_data['E_SALE_AMT']

In [ ]:
esales_data = all_data[['global_id', 'STO_FIN_HRCHY__DCSTAR', 'month', 'E_SALE_AMT','flag_gl_id']]
nonesales_data = all_data[['global_id', 'STO_FIN_HRCHY__DCSTAR', 'month', 'Non-E_SALE_AMT','flag_gl_id']]

In [ ]:
esales_data = esales_data[esales_data['E_SALE_AMT']!=0]
nonesales_data = nonesales_data[nonesales_data['Non-E_SALE_AMT']!=0]

In [ ]:
sales_month = esales_data.groupby('month')['E_SALE_AMT'].sum().reset_index()
sales_month = sales_month.merge(nonesales_data.groupby('month')['Non-E_SALE_AMT'].sum().reset_index(),on='month',how='inner')
sales_month

In [ ]:
esales_df = esales_data.pivot_table(
    index=['global_id','flag_gl_id'],
    columns='month',
    values='E_SALE_AMT',
    aggfunc='sum',
    fill_value=0
)
esales_df = esales_df.reset_index()
esales_df.columns.name = None
# esales_df.rename(columns = {'SOURCE_TYPE':'STO__OWNERSHIP_TYPE_CODE'},inplace=True)
# esales_df['STO__OWNERSHIP_TYPE_CODE'] = esales_df['STO__OWNERSHIP_TYPE_CODE'].replace('COS','COMPANY_OWNED')
# esales_df['STO__OWNERSHIP_TYPE_CODE'] = esales_df['STO__OWNERSHIP_TYPE_CODE'].replace('ISO','INDEPENDENT')

esales_df

In [ ]:
esales_df.columns

In [ ]:
esales_df['global_id'].nunique()

In [ ]:
esales_df.to_parquet(f'Data/esales_pivoted_2025_cleaned_v2.parquet',index=False)

In [ ]:
nonesales_df = nonesales_data.pivot_table(
    index=['global_id','flag_gl_id'],
    columns='month',
    values='Non-E_SALE_AMT',
    aggfunc='sum',
    fill_value=0
)
nonesales_df = nonesales_df.reset_index()
nonesales_df.columns.name = None
# nonesales_df.rename(columns = {'SOURCE_TYPE':'STO__OWNERSHIP_TYPE_CODE'},inplace=True)
# nonesales_df['STO__OWNERSHIP_TYPE_CODE'] = nonesales_df['STO__OWNERSHIP_TYPE_CODE'].replace('COS','COMPANY_OWNED')
# nonesales_df['STO__OWNERSHIP_TYPE_CODE'] = nonesales_df['STO__OWNERSHIP_TYPE_CODE'].replace('ISO','INDEPENDENT')

nonesales_df

In [ ]:
nonesales_df['global_id'].nunique()

In [ ]:
nonesales_df.to_parquet(f'Data/nonesales_pivoted_2025_cleaned_v2.parquet',index=False)